# Exercise 6.3: Geospatial Analysis Using Python

## NYC Healthcare Access Equity Analysis

**Objective:** Use geospatial visualization techniques to explore how health outcomes, socioeconomic factors, and healthcare facility access vary across New York City's neighborhoods and boroughs.

**Data sources:**
- **CDC PLACES (2022):** Census tract-level health outcomes (diabetes, obesity, depression, stroke) for 2,239 matched NYC census tracts
- **American Community Survey (2024):** Borough-level median household income and uninsured rates
- **NY State Health Facility Database:** Latitude and longitude of 2,126 NYC healthcare facilities
- **NYC Open Data (Dept. of City Planning):** 2020 Census Tract boundaries GeoJSON and Borough boundaries GeoJSON

**Hypotheses being tested:**
1. Lower-income boroughs will have significantly higher rates of diabetes, obesity, and stroke
2. Higher healthcare facility density will not correlate with better health outcomes (the "Bronx paradox")
3. Depression will not follow the same income-health pattern as the other chronic conditions

In [3]:
# import libraries and load dataset
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium

master = pd.read_csv('/Users/jessduong/Documents/CF/Achievement 6/02 data/Prepared Data/nyc_health_master.csv')

In [3]:
# Check if geoid format matches census_tract column
print(f"GeoJSON geoid example: {nyc_tracts['features'][0]['properties']['geoid']}")
print(f"Our data census_tract example: {master['census_tract'].iloc[0]}")
print(f"\nGeoJSON geoid type: {type(nyc_tracts['features'][0]['properties']['geoid'])}")
print(f"Our census_tract type: {master['census_tract'].dtype}")

GeoJSON geoid example: 36061000100
Our data census_tract example: 36005000100

GeoJSON geoid type: <class 'str'>
Our census_tract type: int64


In [4]:
# import json
import json

with open('/Users/jessduong/Documents/CF/Achievement 6/02 data/Original Data/NYC_Census_Tracts_2020.geojson', 'r') as f:
    nyc_tracts = json.load(f)

print(f"Type: {nyc_tracts['type']}")
print(f"Number of features: {len(nyc_tracts['features'])}")
print(f"\nSample feature properties:")
print(nyc_tracts['features'][0]['properties'])

Type: FeatureCollection
Number of features: 2325

Sample feature properties:
{':id': 'row-emy8~bcnd-2g7n', ':version': 'rv-ardr_xiih_fjum', ':created_at': '2025-11-24T21:20:25.595Z', ':updated_at': '2025-11-24T21:20:25.595Z', 'ctlabel': '1', 'borocode': '1', 'boroname': 'Manhattan', 'ct2020': '000100', 'boroct2020': '1000100', 'cdeligibil': 'I', 'ntaname': 'The Battery-Governors Island-Ellis Island-Liberty Island', 'nta2020': 'MN0191', 'cdta2020': 'MN01', 'cdtaname': 'MN01 Financial District-Tribeca (CD 1 Equivalent)', 'geoid': '36061000100', 'shape_leng': '10833.0439286', 'shape_area': '1843004.52241'}


In [5]:
# Check if geoid format matches census_tract column
print(f"GeoJSON geoid example: {nyc_tracts['features'][0]['properties']['geoid']}")
print(f"Our data census_tract example: {master['census_tract'].iloc[0]}")
print(f"\nGeoJSON geoid type: {type(nyc_tracts['features'][0]['properties']['geoid'])}")
print(f"Our census_tract type: {master['census_tract'].dtype}")

GeoJSON geoid example: 36061000100
Our data census_tract example: 36005000100

GeoJSON geoid type: <class 'str'>
Our census_tract type: int64


In [7]:
# Convert census_tract to string to match GeoJSON
master['census_tract'] = master['census_tract'].astype(str)

In [8]:
# Clean GeoJSON properties (remove special characters that break folium)
for feature in nyc_tracts['features']:
    geoid = feature['properties']['geoid']
    boroname = feature['properties']['boroname']
    feature['properties'] = {
        'geoid': geoid,
        'boroname': boroname
    }

In [9]:
# Check how many tracts match
geo_ids = set(f['properties']['geoid'] for f in nyc_tracts['features'])
our_ids = set(master['census_tract'].unique())
matches = geo_ids.intersection(our_ids)

In [10]:
print(f"GeoJSON tracts: {len(geo_ids)}")
print(f"Our data tracts: {len(our_ids)}")
print(f"Matching tracts: {len(matches)}")
print(f"In our data but not GeoJSON: {len(our_ids - geo_ids)}")
print(f"In GeoJSON but not our data: {len(geo_ids - our_ids)}")

GeoJSON tracts: 2325
Our data tracts: 2368
Matching tracts: 2239
In our data but not GeoJSON: 129
In GeoJSON but not our data: 86


### Data Matching: GeoJSON and Master Dataset

The 2020 Census Tracts GeoJSON (sourced from NYC Open Data, Department of City Planning) contains 2,325 census tract boundaries. The master dataset contains 2,368 unique census tracts from the CDC PLACES data.

After matching on the census tract FIPS code (geoid), 2,239 tracts matched successfully — a 95% match rate. The 129 tracts in the data but not in the GeoJSON, and the 86 tracts in the GeoJSON but not in our data, are likely due to water-only tracts, parks, or minor boundary changes between census years. These unmatched tracts will appear as gray on the choropleth maps. This is a normal occurrence when working with geographic boundary files and should not affect the overall analysis.

In [11]:
# import mapping libraries
import branca.colormap as cm

In [12]:
# Filter to 2022
df_2022 = master[master['Year'] == 2022].copy()

In [14]:
# Create data lookup
diabetes_dict = df_2022.set_index('census_tract')['diabetes_pct'].to_dict()

In [15]:
# Create colormap
colormap = cm.LinearColormap(
    colors=['#ffffb2', '#fecc5c', '#fd8d3c', '#f03b20', '#bd0026'],
    vmin=df_2022['diabetes_pct'].min(),
    vmax=df_2022['diabetes_pct'].max(),
    caption='Diabetes Rate (%)'
)

In [30]:
# Build diabetes choropleth map
m = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

folium.GeoJson(
    nyc_tracts,
    style_function=lambda feature: {
        'fillColor': colormap(diabetes_dict[feature['properties']['geoid']]) 
            if feature['properties']['geoid'] in diabetes_dict else 'gray',
        'color': 'black',
        'weight': 0.3,
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip(fields=['geoid', 'boroname'], aliases=['Tract:', 'Borough:'])
).add_to(m)

# Save image
colormap.add_to(m)
m.save('/Users/jessduong/Documents/CF/Achievement 6/04 analysis/visualizations/6.3_choropleth maps/diabetes_tract_choropleth.html')
print("Map saved! Open diabetes_tract_choropleth.html in your browser.")

Map saved! Open diabetes_tract_choropleth.html in your browser.


In [31]:
# Obesity choropleth
obesity_dict = df_2022.set_index('census_tract')['obesity_pct'].to_dict()

In [32]:
# Create colormap
colormap_ob = cm.LinearColormap(
    colors=['#ffffb2', '#fecc5c', '#fd8d3c', '#f03b20', '#bd0026'],
    vmin=df_2022['obesity_pct'].min(),
    vmax=df_2022['obesity_pct'].max(),
    caption='Obesity Rate (%)'
)

In [33]:
# Build obesity choropleth map
m2 = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

folium.GeoJson(
    nyc_tracts,
    style_function=lambda feature: {
        'fillColor': colormap_ob(obesity_dict[feature['properties']['geoid']]) 
            if feature['properties']['geoid'] in obesity_dict else 'gray',
        'color': 'black',
        'weight': 0.3,
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip(fields=['geoid', 'boroname'], aliases=['Tract:', 'Borough:'])
).add_to(m2)

# Save image
colormap_ob.add_to(m2)
m2.save('/Users/jessduong/Documents/CF/Achievement 6/04 analysis/visualizations/6.3_choropleth maps/obesity_tract_choropleth.html')
print("Obesity map saved!")

Obesity map saved!


In [34]:
# Depression choropleth
depression_dict = df_2022.set_index('census_tract')['depression_pct'].to_dict()

In [35]:
# Create colormap
colormap_dep = cm.LinearColormap(
    colors=['#ffffb2', '#fecc5c', '#fd8d3c', '#f03b20', '#bd0026'],
    vmin=df_2022['depression_pct'].min(),
    vmax=df_2022['depression_pct'].max(),
    caption='Depression Rate (%)'
)

In [36]:
# Build depression choropleth map
m3 = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

folium.GeoJson(
    nyc_tracts,
    style_function=lambda feature: {
        'fillColor': colormap_dep(depression_dict[feature['properties']['geoid']]) 
            if feature['properties']['geoid'] in depression_dict else 'gray',
        'color': 'black',
        'weight': 0.3,
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip(fields=['geoid', 'boroname'], aliases=['Tract:', 'Borough:'])
).add_to(m3)

# Save image
colormap_dep.add_to(m3)
m3.save('/Users/jessduong/Documents/CF/Achievement 6/04 analysis/visualizations/6.3_choropleth maps/depression_tract_choropleth.html')
print("Depression map saved!")

Depression map saved!


In [37]:
# Stroke choropleth
stroke_dict = df_2022.set_index('census_tract')['stroke_pct'].to_dict()

In [38]:
# Create colormap
colormap_str = cm.LinearColormap(
    colors=['#ffffb2', '#fecc5c', '#fd8d3c', '#f03b20', '#bd0026'],
    vmin=df_2022['stroke_pct'].min(),
    vmax=df_2022['stroke_pct'].max(),
    caption='Stroke Rate (%)'
)

In [39]:
# Build stroke choropleth map
m4 = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

folium.GeoJson(
    nyc_tracts,
    style_function=lambda feature: {
        'fillColor': colormap_str(stroke_dict[feature['properties']['geoid']]) 
            if feature['properties']['geoid'] in stroke_dict else 'gray',
        'color': 'black',
        'weight': 0.3,
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip(fields=['geoid', 'boroname'], aliases=['Tract:', 'Borough:'])
).add_to(m4)

# Save image
colormap_str.add_to(m4)
m4.save('/Users/jessduong/Documents/CF/Achievement 6/04 analysis/visualizations/6.3_choropleth maps/stroke_tract_choropleth.html')
print("Stroke map saved!")

Stroke map saved!


### Exercise 6.3: Geospatial Analysis — Approach

After creating tract-level choropleth maps for all four health outcomes (diabetes, obesity, depression, stroke), the next step is to address the three hypotheses geospatially.

Because income, uninsured rate, and facility density data are available only at the county (borough) level, these variables cannot be mapped at the same tract-level granularity as the health outcomes. To work within this limitation, the approach includes:

1. **Tract-level health outcome choropleths** (completed): Diabetes, obesity, depression, and stroke maps using 2022 CDC PLACES data across 2,239 matched census tracts.
2. **Borough-level socioeconomic choropleths**: Median income, uninsured rate, and facility density mapped at the borough level to support Hypothesis 1 (income predicts chronic disease) and Hypothesis 2 (facility density does not predict outcomes).
3. **Composite health burden index**: A combined score of all four health conditions per tract, providing a single map of overall health need.
4. **Facility point overlay**: Plotting 2,126 individual facility locations on top of the diabetes choropleth to visually show whether facility concentration aligns with health need — directly testing Hypothesis 2 (the Bronx paradox).

## Diabetes choropleth map and facility heatmap for visual comparison of facility clusters and diabetes.

In [41]:
# Prepare to create facility density heatmap
# Load facility data
facilities = pd.read_csv('/Users/jessduong/Documents/CF/Achievement 6/02 data/Original Data/Health_Facility_General_Information.csv')

In [42]:
# Filter to NYC counties
nyc_counties = ['Bronx', 'Kings', 'New York', 'Queens', 'Richmond']
nyc_facilities = facilities[facilities['Facility County'].isin(nyc_counties)].copy()

In [43]:
print(f"NYC facilities: {len(nyc_facilities)}")
print(f"\nColumns with coordinates:")
print([col for col in nyc_facilities.columns if 'lat' in col.lower() or 'lon' in col.lower() or 'geo' in col.lower()])

NYC facilities: 2126

Columns with coordinates:
['Facility Latitude', 'Facility Longitude']


In [51]:
# Standalone facility density heatmap
from folium.plugins import HeatMap

In [52]:
# Create heatmap
m5 = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

heat_data = [[row['Facility Latitude'], row['Facility Longitude']] 
             for _, row in nyc_facilities.iterrows() 
             if pd.notna(row['Facility Latitude']) and pd.notna(row['Facility Longitude'])]

HeatMap(heat_data, radius=10, blur=12, max_zoom=13, min_opacity=0.3).add_to(m5)

# Save image
m5.save('/Users/jessduong/Documents/CF/Achievement 6/04 analysis/visualizations/6.3_choropleth maps/facility_density_heatmap.html')
print("Facility heatmap saved!")

Facility heatmap saved!


## Socioeconomic maps

In [53]:
# Load borough boundaries
with open('/Users/jessduong/Documents/CF/Achievement 6/02 data/Original Data/NYC_Borough_Boundaries.geojson', 'r') as f:
    borough_geo = json.load(f)

In [54]:
# Check properties
print(borough_geo['features'][0]['properties'])

{':id': 'row-mvnh~z8ju-gq5z', ':version': 'rv-nau4_7c46-8i7d', ':created_at': '2025-11-24T21:41:25.965Z', ':updated_at': '2025-11-24T21:41:25.965Z', 'borocode': '5', 'boroname': 'Staten Island', 'shape_area': '1623618358.46', 'shape_leng': '325912.288988'}


In [55]:
# Clean borough GeoJSON properties
for feature in borough_geo['features']:
    borocode = feature['properties']['borocode']
    boroname = feature['properties']['boroname']
    feature['properties'] = {
        'borocode': borocode,
        'boroname': boroname
    }

In [56]:
# Create borough-level data for income and uninsured
borough_data = df_2022.groupby('county').agg({
    'median_income': 'first',
    'pct_uninsured': 'first',
    'facility_count': 'first'
}).reset_index()

In [57]:
print(borough_data)

     county  median_income  pct_uninsured  facility_count
0     Bronx        48676.0            7.1           440.0
1     Kings        80263.0            5.6           676.0
2  New York       103931.0            4.0           453.0
3    Queens        86136.0            7.4           449.0
4  Richmond        98333.0            4.4           108.0


In [59]:
 # Map county names to borough names
county_to_boro = {
    'Bronx': 'Bronx',
    'Kings': 'Brooklyn',
    'New York': 'Manhattan',
    'Queens': 'Queens',
    'Richmond': 'Staten Island'
}

In [60]:
borough_data['boroname'] = borough_data['county'].map(county_to_boro)

In [61]:
# Median Income choropleth
income_dict = borough_data.set_index('boroname')['median_income'].to_dict()

In [63]:
# Create colormap
colormap_inc = cm.LinearColormap(
    colors=['#ffffb2', '#fecc5c', '#fd8d3c', '#f03b20', '#bd0026'],
    vmin=borough_data['median_income'].min(),
    vmax=borough_data['median_income'].max(),
    caption='Median Household Income ($)'
)

m6 = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

In [64]:
# Build map
folium.GeoJson(
    borough_geo,
    style_function=lambda feature: {
        'fillColor': colormap_inc(income_dict[feature['properties']['boroname']]) 
            if feature['properties']['boroname'] in income_dict else 'gray',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip(fields=['boroname'], aliases=['Borough:'])
).add_to(m6)

# Save image
colormap_inc.add_to(m6)
m6.save('/Users/jessduong/Documents/CF/Achievement 6/04 analysis/visualizations/6.3_choropleth maps/income_borough_choropleth.html')
print("Income map saved!")

Income map saved!


In [65]:
# Uninsured rate choropleth
uninsured_dict = borough_data.set_index('boroname')['pct_uninsured'].to_dict()

In [66]:
# Create colormap
colormap_un = cm.LinearColormap(
    colors=['#ffffb2', '#fecc5c', '#fd8d3c', '#f03b20', '#bd0026'],
    vmin=borough_data['pct_uninsured'].min(),
    vmax=borough_data['pct_uninsured'].max(),
    caption='Uninsured Rate (%)'
)

m7 = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

In [67]:
# Build map
folium.GeoJson(
    borough_geo,
    style_function=lambda feature: {
        'fillColor': colormap_un(uninsured_dict[feature['properties']['boroname']]) 
            if feature['properties']['boroname'] in uninsured_dict else 'gray',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip(fields=['boroname'], aliases=['Borough:'])
).add_to(m7)

# Save image
colormap_un.add_to(m7)
m7.save('/Users/jessduong/Documents/CF/Achievement 6/04 analysis/visualizations/6.3_choropleth maps/uninsured_borough_choropleth.html')
print("Uninsured map saved!")

Uninsured map saved!


## Create health burden index (tract-level map that combines all four conditions into one score.)

In [68]:
# Create a composite Health Burden Index that combines all 4 health conditions
# into a single score per census tract (0 = healthiest, 100 = most burdened)

from sklearn.preprocessing import MinMaxScaler

In [69]:
# Select only columns needed
df_index = df_2022[['census_tract', 'diabetes_pct', 'obesity_pct', 'depression_pct', 'stroke_pct']].copy()

In [70]:
# Drop rows with missing values (depression is missing for some tracts)
df_index = df_index.dropna()

In [71]:
# Normalize each health condition to a 0-1 scale so they contribute equally
scaler = MinMaxScaler()
df_index[['diab_norm', 'obes_norm', 'dep_norm', 'stroke_norm']] = scaler.fit_transform(
    df_index[['diabetes_pct', 'obesity_pct', 'depression_pct', 'stroke_pct']]
)

In [72]:
# Average the 4 normalized scores and scale to 0-100
df_index['health_burden'] = df_index[['diab_norm', 'obes_norm', 'dep_norm', 'stroke_norm']].mean(axis=1) * 100

In [73]:
# Create lookup dictionary for the map
burden_dict = df_index.set_index('census_tract')['health_burden'].to_dict()

In [74]:
# Build colormap
colormap_hb = cm.LinearColormap(
    colors=['#ffffb2', '#fecc5c', '#fd8d3c', '#f03b20', '#bd0026'],
    vmin=df_index['health_burden'].min(),
    vmax=df_index['health_burden'].max(),
    caption='Health Burden Index (0-100)'
)

In [75]:
# Build map
m8 = folium.Map(location=[40.7128, -74.0060], zoom_start=10)

folium.GeoJson(
    nyc_tracts,
    style_function=lambda feature: {
        'fillColor': colormap_hb(burden_dict[feature['properties']['geoid']]) 
            if feature['properties']['geoid'] in burden_dict else 'gray',
        'color': 'black',
        'weight': 0.3,
        'fillOpacity': 0.7
    },
    tooltip=folium.GeoJsonTooltip(fields=['geoid', 'boroname'], aliases=['Tract:', 'Borough:'])
).add_to(m8)

# Save image"
colormap_hb.add_to(m8)
m8.save('/Users/jessduong/Documents/CF/Achievement 6/04 analysis/visualizations/6.3_choropleth maps/health_burden_index_choropleth.html')
print("Health burden index map saved!")

Health burden index map saved!


## Exercise 6.3: Geospatial Analysis — Results and Discussion

### Note on AI Usage
AI (Claude) was used in this exercise to reassess the project structure after tutor feedback indicated the original 5-row county-level dataset did not meet the 1,500-row minimum requirement. AI assisted in thinking through how to restructure the data to census tract level while keeping the NYC healthcare topic, evaluating the trade-offs between different mapping approaches (tract-level vs. borough-level), and deciding which combination of visualizations would best address the three hypotheses given the mixed granularity of the data (tract-level health outcomes but county-level socioeconomic variables).

---

### Approach

The geospatial analysis uses two levels of geographic detail depending on data availability. Health outcome data (diabetes, obesity, depression, stroke) from CDC PLACES is available at the census tract level (2,239 matched tracts), allowing for detailed neighborhood-level mapping. Socioeconomic data (income, uninsured rate) and facility counts are only available at the borough level, so these are mapped as 5 borough-level choropleths. A facility density heatmap was created using the latitude and longitude of all 2,126 NYC healthcare facilities. A composite Health Burden Index was also created to provide a single summary view of overall health need per tract.

The 2020 Census Tract boundaries GeoJSON was sourced from NYC Open Data (Department of City Planning). Of the 2,368 unique census tracts in the master dataset, 2,239 (95%) matched to the GeoJSON. Unmatched tracts appear as gray on the maps and are likely water-only tracts, parks, or minor boundary changes between census years.

---

### Maps Created and Key Findings

**Tract-level choropleths (2,239 census tracts, 2022 data):**

1. **Diabetes Rate**: Highest concentrations in the Bronx and central Brooklyn, with rates above 30% in some tracts. Manhattan consistently shows the lowest rates, under 10% in most tracts. Significant within-borough variation is visible — Queens ranges from under 10% to above 40% depending on the tract.

2. **Obesity Rate**: Follows a similar geographic pattern to diabetes. The Bronx shows the darkest shading, with rates above 40% in many tracts. Manhattan tracts cluster below 25%. Staten Island shows pockets of high obesity despite higher borough-level income.

3. **Depression Rate**: This map breaks the pattern seen in the other three conditions. Manhattan shows the highest depression rates, above 20% in most tracts, while Queens has the lowest. This is the opposite of what the diabetes, obesity, and stroke maps show, and visually confirms that depression is driven by different factors.

4. **Stroke Rate**: Closely mirrors the diabetes map, consistent with the strong correlation (r = 0.90) found in Exercise 6.2. The Bronx has the highest stroke rates and Manhattan the lowest.

5. **Health Burden Index (0-100)**: A composite score created by normalizing all four health conditions to a 0-1 scale using MinMaxScaler, averaging them, and scaling to 0-100. The Bronx and parts of Brooklyn show the highest overall health burden (scores above 50), while Manhattan tracts score lowest. This map provides a single summary view of where health need is greatest across all conditions.

**Borough-level choropleths:**

6. **Median Household Income**: Manhattan ($103,931) is the highest and the Bronx ($48,676) the lowest. The visual contrast between these two boroughs directly mirrors the inverse pattern seen in the diabetes, obesity, and stroke maps.

7. **Uninsured Rate**: Queens (7.4%) and the Bronx (7.1%) have the highest uninsured rates. Manhattan (4.0%) and Staten Island (4.4%) have the lowest. This aligns with the income pattern — lower-income boroughs also have less insurance coverage.

**Facility analysis:**

8. **Facility Density Heatmap**: The highest concentration of healthcare facilities is in Manhattan and the Bronx, with a visible hotspot in midtown Manhattan. Queens and Staten Island have noticeably fewer facilities. This is significant because the Bronx has both the highest facility density and the worst health outcomes.

---

### Answering Research Questions

**Hypothesis 1 — Income predicts chronic disease: SUPPORTED.** Comparing the income map to the diabetes, obesity, and stroke maps shows a clear inverse relationship. The Bronx (lowest income at $48,676) has the worst outcomes, and Manhattan (highest income at $103,931) has the best. The uninsured rate map reinforces this — boroughs with less insurance coverage also have higher chronic disease rates. The geographic patterns are consistent across all three chronic conditions (diabetes, obesity, stroke), suggesting income is a systemic driver of health outcomes rather than a condition-specific factor.

**Hypothesis 2 — Facility density does not predict health outcomes: SUPPORTED.** The facility heatmap shows the Bronx has one of the highest concentrations of healthcare facilities in the city, yet its health outcomes are the worst across diabetes, obesity, and stroke. This confirms the "Bronx paradox" — more facilities alone do not lead to better health. Meanwhile, Queens has the fewest facilities but moderate health outcomes. This suggests that healthcare facility access is not the primary driver of health equity, and that upstream factors like income and insurance coverage play a larger role.

**Hypothesis 3 — Depression is driven by different factors: SUPPORTED.** The depression choropleth visually reverses the pattern seen in the other three health maps. Manhattan has the highest depression rates despite having the highest income and lowest rates of diabetes, obesity, and stroke. This suggests that depression in NYC is influenced by non-economic urban stressors — such as cost of living pressure, social isolation, population density, or lifestyle factors — rather than the income-related drivers behind other chronic conditions.

---

### New Research Questions

1. **What specific urban stressors drive Manhattan's high depression rates?** Is it cost of living pressure, social isolation, long work hours, or population density? Tract-level mental health service data and housing cost data could help investigate this.

2. **Why doesn't facility density improve outcomes in the Bronx?** Are the facilities the right type (e.g., primary care vs. specialty)? Are there barriers beyond physical access, such as wait times, language barriers, or insurance acceptance? A facility-type breakdown could provide answers.

3. **What would tract-level income data reveal?** The current income data is borough-level, which masks within-borough variation. The box plots in Exercise 6.2 showed wide ranges of health outcomes within each borough. Census tract-level income data from the American Community Survey could uncover hidden pockets of need within wealthier boroughs and explain some of the within-borough variation visible in the choropleth maps.

4. **How do income and insurance interact?** Queens has moderate income but the highest uninsured rate. Does the combination of low insurance and moderate income create a compounding effect on health outcomes that neither variable explains alone?